In [11]:
import pandas as pd
pd.set_option('display.max_columns', None)
import sqlite3
import glob
import os

# 1. Data Ingestion

Loading raw CSV files from Date/ directory, extracting year and month from filenames to create a date column and conatenating all files into a single DataFrame

In [4]:
files = glob.glob('Data/*.csv')
df_list = []
for f in files:
    df = pd.read_csv(f)
    filename = os.path.basename(f)
    parts = filename.replace('.csv','').split('_')
    year = int(parts[3])
    month = int(parts[4])
    df['date'] = pd.to_datetime(f'{year}-{month}-01')
    df_list.append(df)
df = pd.concat(df_list, ignore_index=False)

# 2. Data Loading to SQLite

Connecting to the SQLite database and storing the merged dataset in a table named Rent for efficient SQL-based exploration

In [6]:
conn = sqlite3.connect('Rent.db')
df.to_sql('Rent', conn, if_exists='replace',index=False)

70847

# 3. Initial Data Exploration & Profiling

Performing an initial audit of the dataset to indentity missing values, data types, and potential outliers

## 3.1 Query Helper Function

In [7]:
def run_query(query):
    return pd.read_sql_query(query,conn)

## 3.2 Phase I: Data Integrity & Null Analysis

Here I will audit the raw dataset to identity missing values and check the uniqueness of record

### 3.2.1 Data Types & Schema Overview

Inspect the table schema to ensure all columns have the correct data types

In [8]:
run_query("PRAGMA table_info('Rent')")

,cid,name,type,notnull,dflt_value,pk
0,0,id,TEXT,0,None,0
1,1,city,TEXT,0,None,0
2,2,type,TEXT,0,None,0
3,3,squareMeters,REAL,0,None,0
4,4,rooms,REAL,0,None,0
5,5,floor,REAL,0,None,0
6,6,floorCount,REAL,0,None,0
7,7,buildYear,REAL,0,None,0
8,8,latitude,REAL,0,None,0
9,9,longitude,REAL,0,None,0


**Decision:** All data types are correct

### 3.2.2 Null Value Percentage Check

Calculate the precentage of missing values for each column to identity unreliable data.

In [12]:
run_query("""
SELECT 
ROUND(100* (COUNT(*) - COUNT(id)) / COUNT(*), 2) as id_null_pct,
ROUND(100* (COUNT(*) - COUNT(city)) / COUNT(*), 2) as city_null_pct,
ROUND(100* (COUNT(*) - COUNT(type)) / COUNT(*), 2) as type_null_pct,
ROUND(100* (COUNT(*) - COUNT(squareMeters)) / COUNT(*), 2) as squareMeters_null_pct,
ROUND(100* (COUNT(*) - COUNT(rooms)) / COUNT(*), 2) as rooms_null_pct,
ROUND(100* (COUNT(*) - COUNT(floor)) / COUNT(*), 2) as floor_null_pct,
ROUND(100* (COUNT(*) - COUNT(floorCount)) / COUNT(*), 2) as floorCount_null_pct,
ROUND(100* (COUNT(*) - COUNT(buildYear)) / COUNT(*), 2) as buildYear_null_pct,
ROUND(100* (COUNT(*) - COUNT(latitude)) / COUNT(*), 2) as latitude_null_pct,
ROUND(100* (COUNT(*) - COUNT(longitude)) / COUNT(*), 2) as longitude_null_pct,
ROUND(100 * (COUNT(*) - COUNT(centreDistance)) / COUNT(*),2) as centreDistance_null_pct,
ROUND(100 * (COUNT(*) - COUNT(poiCount)) / COUNT(*),2) as poiCount_null_pct,
ROUND(100 * (COUNT(*) - COUNT(schoolDistance)) / COUNT(*),2) as schoolDistance_null_pct,
ROUND(100 * (COUNT(*) - COUNT(clinicDistance)) / COUNT(*),2) as clinicDistance_null_pct,
ROUND(100 * (COUNT(*) - COUNT(postOfficeDistance)) / COUNT(*),2) as postOfficeDistance_null_pct,
ROUND(100 * (COUNT(*) - COUNT(kindergartenDistance)) / COUNT(*),2) as kindergartenDistance_null_pct,
ROUND(100 * (COUNT(*) - COUNT(restaurantDistance)) / COUNT(*),2) as restaurantDistance_null_pct,
ROUND(100 * (COUNT(*) - COUNT(collegeDistance)) / COUNT(*),2) as collegeDistance_null_pct,
ROUND(100 * (COUNT(*) - COUNT(pharmacyDistance)) / COUNT(*),2) as pharmacyDistance_null_pct,
ROUND(100* (COUNT(*) - COUNT(ownership)) / COUNT(*), 2) as ownership_null_pct,
ROUND(100* (COUNT(*) - COUNT(buildingMaterial)) / COUNT(*), 2) as buildingMaterial_null_pct,
ROUND(100* (COUNT(*) - COUNT(condition)) / COUNT(*), 2) as condition_null_pct,
ROUND(100* (COUNT(*) - COUNT(hasParkingSpace)) / COUNT(*), 2) as hasParkingSpace_null_pct,
ROUND(100* (COUNT(*) - COUNT(hasBalcony)) / COUNT(*), 2) as hasBalcony_null_pct,
ROUND(100* (COUNT(*) - COUNT(hasElevator)) / COUNT(*), 2) as hasElevator_null_pct,
ROUND(100* (COUNT(*) - COUNT(hasSecurity)) / COUNT(*), 2) as hasSecurity_null_pct,
ROUND(100* (COUNT(*) - COUNT(hasStorageRoom)) / COUNT(*), 2) as hasStorageRoom_null_pct,
ROUND(100* (COUNT(*) - COUNT(price)) / COUNT(*), 2) as price_null_pct,
ROUND(100* (COUNT(*) - COUNT(date)) / COUNT(*), 2) as Date_null_pct
FROM Rent 
"""
)

,id_null_pct,city_null_pct,type_null_pct,squareMeters_null_pct,rooms_null_pct,floor_null_pct,floorCount_null_pct,buildYear_null_pct,latitude_null_pct,longitude_null_pct,centreDistance_null_pct,poiCount_null_pct,schoolDistance_null_pct,clinicDistance_null_pct,postOfficeDistance_null_pct,kindergartenDistance_null_pct,restaurantDistance_null_pct,collegeDistance_null_pct,pharmacyDistance_null_pct,ownership_null_pct,buildingMaterial_null_pct,condition_null_pct,hasParkingSpace_null_pct,hasBalcony_null_pct,hasElevator_null_pct,hasSecurity_null_pct,hasStorageRoom_null_pct,price_null_pct,Date_null_pct
0,0.0,0.0,23.0,0.0,0.0,12.0,1.0,27.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,40.0,72.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0


**Decision:** I decided to remove two columns **buildingMaterial** (40% missing values) and **condition** (72% missing values).
The rest of columns with missing data will be filled in the next part of analysis

### 3.2.1 Duplicated ID Check

Verify duplicated IDs and check if they represent price changes of the same apartment over time.

In [15]:
run_query("SELECT COUNT(*) - COUNT(DISTINCT id) as duplicate_id FROM Rent")

,duplicate_id
0,32906


In [20]:
run_query("""SELECT id, COUNT(DISTINCT price) as change
FROM Rent
GROUP By id
HAVING change > 1
ORDER By change DESC
""")

,id,change
0,992d88fc0d5b0f35487a412ed3b3cf39,7
1,af906c9d6e8b89eff9aee76dce675f46,6
2,804020ca1176e47e7d31ee4e8ea70ae4,6
3,624defa0372419ec0ee86dfae350213f,6
4,42bb30c0ce11c159b733164e0615147b,6
...,...,...
5055,0057c21870db2e1a7599d748e8e7286c,2
5056,0035f80ff649fb857a93e9154b7cf6ea,2
5057,0033c3926483002f56be13572f0afe13,2
5058,001caa8a8793a640f2bb73d29ed7104c,2


**Decision:** I decided to keep all duplicated records because the dataset contains 5060 apartments that changed their price between 2 and 7 times. This hitorical data will be valuable for time-series analysis.

## 3.3 Phase II: Feature Engineering & Description Statistics

In this phase I add necessary columns and perform a high-level statistical overview

### 3.3.1 Feature Engineering: Price Per Square Meter

Calculate a new column price_per_m2 to facilitate outilers detection.

In [21]:
cursor = conn.cursor()

In [22]:
cursor.execute("ALTER TABLE Rent ADD COLUMN price_per_m2 REAL DEFAULT 0.0")
conn.commit()

In [23]:
cursor.execute("UPDATE Rent SET price_per_m2 = ROUND((price / squareMeters),2)")
conn.commit()

In [24]:
run_query("SELECT * FROM Rent LIMIT 2")

,id,city,type,squareMeters,rooms,floor,floorCount,buildYear,latitude,longitude,centreDistance,poiCount,schoolDistance,clinicDistance,postOfficeDistance,kindergartenDistance,restaurantDistance,collegeDistance,pharmacyDistance,ownership,buildingMaterial,condition,hasParkingSpace,hasBalcony,hasElevator,hasSecurity,hasStorageRoom,price,date,price_per_m2
0,2a1a6db97ff122d6bc148abb6f0e498a,szczecin,blockOfFlats,52.0,2.0,3.0,3.0,2008.0,53.460535,14.545416,4.26,2.0,0.753,1.049,0.595,0.674,0.229,2.200,0.307,condominium,brick,None,yes,yes,no,no,no,3500,2023-11-01 00:00:00,67.31
1,5a0cb42c088eadf75aa98a4118640048,szczecin,blockOfFlats,57.0,3.0,NaN,10.0,1976.0,53.440955,14.566024,1.98,14.0,0.126,0.875,0.586,0.058,0.129,1.323,0.189,condominium,None,premium,no,yes,yes,no,no,3000,2023-11-01 00:00:00,52.63


New column added successfully

### 3.3.2 Statistical Summary

Calculate the Min, Max, Q1, Q2, Q3, Mean, for both price and price_per_m2 across cities.

#### 3.3.2.1 Price

In [27]:
run_query("""
WITH RankedPrice AS(
SELECT price,city,
ROW_NUMBER() OVER (PARTITION By city ORDER By price) as rnk,
COUNT(*) OVER (PARTITION By city) as total
FROM Rent

)
SELECT city, 
AVG(price) FILTER (WHERE rnk BETWEEN (total + 3) / 4 AND (total + 4) / 4 ) as Q1,
AVG(price) FILTER (WHERE rnk BETWEEN (total + 1) / 2 AND (total + 2) / 2) as Q2,
AVG(price) FILTER (WHERE rnk BETWEEN (3 * total + 1) / 4 AND (3 * total + 4) / 4) as Q3,
ROUND(AVG(price),2) as Mean,
ROUND(MAX(price),2) as price_max,
ROUND(MIN(price),2) as price_min
FROM RankedPrice
GROUP By city
""")

,city,Q1,Q2,Q3,Mean,price_max,price_min
0,bialystok,1700.0,2000.0,2400.0,2032.56,3600.0,780.0
1,bydgoszcz,1600.0,1990.0,2400.0,2081.41,4500.0,1000.0
2,czestochowa,1500.0,1900.0,2300.0,1935.52,3700.0,980.0
3,gdansk,2500.0,2950.0,3600.0,3237.52,7800.0,1800.0
4,gdynia,2300.0,2800.0,3900.0,3191.00,8000.0,1500.0
5,katowice,1800.0,2300.0,2950.0,2464.39,6500.0,346.0
6,krakow,2500.0,3000.0,3700.0,3340.18,10000.0,1700.0
7,lodz,1650.0,2000.0,2500.0,2120.30,4900.0,1000.0
8,lublin,2000.0,2500.0,2950.0,2526.28,4900.0,1400.0
9,poznan,2000.0,2400.0,2900.0,2589.38,6000.0,1250.0


**Insights:** The statistical summary of **price** reveals two potencial outliers: an unusually high value 23000zł in Warsaw and an extremely low value 346zł in Katowice. These observations will investigated futher in the nest steps of the analysis.

#### 3.3.2.1 Price Per Square Meter

In [31]:
run_query("""
WITH RankedPrice AS(
SELECT price_per_m2,city,
ROW_NUMBER() OVER (PARTITION By city ORDER By price_per_m2) as rnk,
COUNT(*) OVER (PARTITION By city) as total
FROM Rent

)
SELECT city, 
AVG(price_per_m2) FILTER (WHERE rnk BETWEEN (total + 3) / 4 AND (total + 4) / 4 ) as Q1,
AVG(price_per_m2) FILTER (WHERE rnk BETWEEN (total + 1) / 2 AND (total + 2) / 2) as Q2,
AVG(price_per_m2) FILTER (WHERE rnk BETWEEN (3 * total + 1) / 4 AND (3 * total + 4) / 4) as Q3,
ROUND(AVG(price_per_m2),2) as Mean,
ROUND(MAX(price_per_m2),2) as price_per_m2_max,
ROUND(MIN(price_per_m2),2) as price_per_m2_min
FROM RankedPrice
GROUP By city
""")

,city,Q1,Q2,Q3,Mean,price_per_m2_max,price_per_m2_min
0,bialystok,35.42,43.195,50.00,42.94,66.67,13.00
1,bydgoszcz,37.50,45.240,55.56,46.57,73.53,25.37
2,czestochowa,31.94,38.890,48.91,40.28,66.19,12.10
3,gdansk,57.45,66.540,76.86,67.77,118.64,40.00
4,gdynia,47.52,55.560,68.63,58.65,109.06,33.47
5,katowice,38.64,48.330,59.01,47.98,88.89,11.30
6,krakow,55.39,65.120,76.67,66.95,115.63,38.06
7,lodz,38.20,45.240,53.78,46.45,80.90,23.33
8,lublin,41.56,51.020,61.90,52.30,92.19,26.64
9,poznan,44.62,52.515,62.34,55.39,113.02,24.07


**Insights:** The statistical summary of **price_per_m2** shows a few possible outliers. Cities with a minimum below 30 PLN per square meter might have a wrong value. High prices in big cities are normal. We will check in the next steps how many of these listings exist and remove them if they are clearly errors.

### 3.3.3 Quantivative Range Check

Check Min and Max values for rest numeric columns

In [34]:
run_query("""
SELECT 
MAX(squareMeters) as squareMeters_max,
MIN(squareMeters) as squareMeters_min,
MAX(rooms) as rooms_max,
MIN(rooms) as rooms_min,
MAX(floor) as floor_max,
MIN(floor) as floor_min,
MAX(floorCount) as floorCount_max,
MIN(floorCount) as floorCount_min,
MAX(buildYear) as buildYear_max,
MIN(buildYear) as buildYear_min,
MAX(centreDistance) as centreDistance_max,
MIN(centreDistance) as centreDistance_min,
MAX(poiCount) as poiCount_max,
MIN(poiCount) as poiCount_min,
MAX(schoolDistance) as schoolDistance_max,
MIN(schoolDistance) as schoolDistance_min,
MAX(clinicDistance) as clinicDistance_max,
MIN(clinicDistance) as clinicDistance_min,
MAX(postOfficeDistance) as postOfficeDistance_max,
MIN(postOfficeDistance) as postOfficeDistance_min,
MAX(kindergartenDistance) as kindergartenDistance_max,
MIN(kindergartenDistance) as kindergartenDistance_min,
MAX(restaurantDistance) as restaurantDistance_max,
MIN(restaurantDistance) as restaurantDistance_min,
MAX(collegeDistance) as collegeDistance_max,
MIN(collegeDistance) as collegeDistance_min,
MAX(pharmacyDistance) as pharmacyDistance_max,
MIn(pharmacyDistance) as pharmacyDistance_min
FROM Rent
""")

,squareMeters_max,squareMeters_min,rooms_max,rooms_min,floor_max,floor_min,floorCount_max,floorCount_min,buildYear_max,buildYear_min,centreDistance_max,centreDistance_min,poiCount_max,poiCount_min,schoolDistance_max,schoolDistance_min,clinicDistance_max,clinicDistance_min,postOfficeDistance_max,postOfficeDistance_min,kindergartenDistance_max,kindergartenDistance_min,restaurantDistance_max,restaurantDistance_min,collegeDistance_max,collegeDistance_min,pharmacyDistance_max,pharmacyDistance_min
0,150.0,25.0,6.0,1.0,30.0,1.0,30.0,1.0,2024.0,1850.0,16.62,0.02,210.0,0.0,4.856,0.002,4.99,0.001,4.939,0.001,4.751,0.001,4.961,0.001,5.0,0.004,4.986,0.001


**Insights:** The min and max values of the remaining numerical features do not indicate anomalies. Observations such as 210 or 0 POIs, very small distances, and building years as early as 1850 are plausible and consistent with real-world data.

## 3.4 Phase III: Quality Audit & Business Rules

Here i look for anomalies and logical errors in the dataset based on domain knowledge.

### 3.4.1 Categorical Standarization Check

Verify unique values for categorical columns.

In [37]:
run_query("""
SELECT DISTINCT 'city' AS column_city, city AS value
FROM Rent
UNION ALL
SELECT DISTINCT 'type', type FROM Rent
UNION ALL
SELECT DISTINCT 'ownership', ownership FROM Rent
UNION ALL
SELECT DISTINCT 'hasParkingSpace', hasParkingSpace FROM Rent
UNION ALL
SELECT DISTINCT 'hasBalcony', hasBalcony FROM Rent
UNION ALL 
SELECT DISTINCT 'hasElevator', hasElevator FROM Rent
UNION ALL 
SELECT DISTINCT 'hasSecurity', hasSecurity FROM Rent
UNION ALL
SELECT DISTINCT 'hasStorageRoom', hasStorageRoom FROM Rent
""")

,column_city,value
0,city,szczecin
1,city,gdynia
2,city,krakow
3,city,poznan
4,city,bialystok
5,city,gdansk
6,city,wroclaw
7,city,radom
8,city,rzeszow
9,city,lodz


**Insights:** All categorical data are correct.

### 3.4.2 Logical Consistency Check

Identify records where apartment floor is higher than total building floor count

In [38]:
run_query("""SELECT floor, floorCount
FROM Rent 
WHERE floor > floorCount
""")

,floor,floorCount


**Insights:** We got 0 records where floor is higher than total building floor count.

### 3.4.3 Price anomaly Detection

Flag apartments with price per square meter <= 30 PLN

In [44]:
run_query("""
SELECT id,city, squareMeters,rooms,price,price_per_m2
FROM Rent 
WHERE price_per_m2 <= 30
ORDER By price_per_m2 
"""
         )

,id,city,squareMeters,rooms,price,price_per_m2
0,57cf16567b54e67fe663fbe09ec7ce1c,katowice,40.89,2.0,462,11.3
1,5c3e9c26821fd7687d155f1cb8368bad,katowice,41.24,2.0,466,11.3
2,6b6fec05ccaacf915f1c20a77bb81b99,katowice,41.23,2.0,466,11.3
3,9bbbb31792060a7d6ce48ff75242fd00,katowice,41.15,2.0,465,11.3
4,94d9245e8863119edf794fe596a14d57,katowice,43.80,2.0,495,11.3
...,...,...,...,...,...,...
841,fec8e1cfa928e98d2d4a01e1c35ae412,czestochowa,90.00,4.0,2700,30.0
842,f9b369f92e99de5bad87b3967f87956a,czestochowa,50.00,2.0,1500,30.0
843,eef9564b0ca476155203c8e32d85baf3,bydgoszcz,50.00,2.0,1500,30.0
844,813a5a0ce6ba8a4c7cb286469e243fdb,bydgoszcz,50.00,2.0,1500,30.0


**Insights:** After checking price_per_m2 <= 30 PLN I found some listings like 90 m² apartment for 2700 PLN are actually valid. This means not all low values are errors, but we still remove the clear mistakes in next step of analysis.

## 3.5 Phase IV: Data Cleaning & Export

Create the Rent_Stage_1 table by dropping columns with excessive mising values **buildgMaterials** and **condition**. All other anomaly filtering like price outliers will be deferred to the EDA phase.

In [45]:
query_create_stage_1 = """
CREATE TABLE Rent_Stage_1 AS
SELECT id,date,city,type,squareMeters,rooms,floor,floorCount,buildYear,latitude,longitude,centreDistance,poiCount,schoolDistance,
clinicDistance,postOfficeDistance,kindergartenDistance,restaurantDistance,collegeDistance,pharmacyDistance,ownership,
hasParkingSpace,hasBalcony,hasElevator,hasSecurity,hasStorageRoom,price
FROM Rent
"""

In [46]:
conn.execute("DROP TABLE IF EXISTS Rent_Stage_1")
conn.execute(query_create_stage_1)
conn.commit()

In [47]:
run_query("PRAGMA table_info('Rent_Stage_1')")

,cid,name,type,notnull,dflt_value,pk
0,0,id,TEXT,0,None,0
1,1,date,NUM,0,None,0
2,2,city,TEXT,0,None,0
3,3,type,TEXT,0,None,0
4,4,squareMeters,REAL,0,None,0
5,5,rooms,REAL,0,None,0
6,6,floor,REAL,0,None,0
7,7,floorCount,REAL,0,None,0
8,8,buildYear,REAL,0,None,0
9,9,latitude,REAL,0,None,0
